In [220]:
import pandas as pd
import csv
import chardet
import re
import shutil
import os
import glob
import csv
from difflib import SequenceMatcher

CleanData to initialData

In [ ]:
# Read the CSV file
MAIN_DF = pd.read_csv("cleanData11-12-01.csv")
df_wydatki = MAIN_DF.copy()

# Remove unnecessary columns
cols = ["Data księgowania", "Tytuł", "Nr rachunku", "Nazwa banku", "Szczegóły", 
        "Nr transakcji", "Waluta", "Waluta.1", "Kwota płatności w walucie", 
        "Waluta.2", "Konto", "Saldo po transakcji", "Waluta.3"]

# Drop columns safely using list comprehension for existing columns only
existing_cols = [col for col in cols if col in df_wydatki.columns]
df_wydatki.drop(columns=existing_cols, inplace=True)

# Rename column - fixed by assigning the result back to df_wydatki
df_wydatki = df_wydatki.rename(columns={'Kwota transakcji (waluta rachunku)': 'Kwota transakcji'})

# Remove last 3 rows
df_wydatki = df_wydatki.iloc[:-3]

# Consolidate transaction amount columns
# First ensure the column exists and contains numeric values
df_wydatki['Kwota transakcji'] = df_wydatki['Kwota transakcji'].str.replace(',', '.').astype(float)
if 'Kwota blokady/zwolnienie blokady' in df_wydatki.columns:
    df_wydatki['Kwota blokady/zwolnienie blokady'] = df_wydatki['Kwota blokady/zwolnienie blokady'].str.replace(',', '.').astype(float)
    df_wydatki['Kwota transakcji'] = df_wydatki[['Kwota transakcji', 'Kwota blokady/zwolnienie blokady']].sum(axis=1)
    df_wydatki.drop(columns=['Kwota blokady/zwolnienie blokady'], inplace=True)

# Convert contractor names to uppercase
df_wydatki["Dane kontrahenta"] = df_wydatki["Dane kontrahenta"].str.upper()

# Sort by contractor name and reset index
df_wydatki.sort_values("Dane kontrahenta", inplace=True)
df_wydatki.reset_index(drop=True, inplace=True)

# Export to CSV
df_wydatki.to_csv("initialData11-12-01.csv", sep=',', encoding='utf-8', index=False)

# Display the result
print(df_wydatki)


In [ ]:
initialData to consolidatedData

In [231]:
# Read the CSV file
df_wydatki_consolidated = pd.read_csv("initialData11-12-01.csv")

# Make sure column names are correct and in the same row
# If needed, reset the header row
df_wydatki_consolidated.columns = df_wydatki_consolidated.columns.str.strip()

# Convert Kwota transakcji to numeric, handling any potential string/formatting issues
df_wydatki_consolidated['Kwota transakcji'] = pd.to_numeric(df_wydatki_consolidated['Kwota transakcji'].astype(str).str.replace(',', '.'), errors='coerce')

# Group by Dane kontrahenta and sum the values
df_wydatki_consolidated = df_wydatki_consolidated.groupby('Dane kontrahenta', as_index=False)['Kwota transakcji'].sum()

# Reset index if needed
df_wydatki_consolidated = df_wydatki_consolidated.reset_index(drop=True)






getRowIndexes = []
for row in df_wydatki_consolidated["Dane kontrahenta"].index:
    rowIndexes.append(row)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127]



def getIndex():
    print(df_wydatki_consolidated["Dane kontrahenta"].index)

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

for value in df_wydatki_consolidated["Dane kontrahenta"]:
    if df_wydatki_consolidated["Dane kontrahenta"].index > 0:
        similar(a, b)


print(df_wydatki_consolidated["Dane kontrahenta"].index)